<a href="https://colab.research.google.com/github/ProfDoof/advent_of_code/blob/2022/day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 

## Load Data

In [1]:
from pathlib import Path

DAY = 7
DATA_FILE = Path.cwd() / 'drive' / 'MyDrive' / 'AdventOfCode' / 'aoc_data' / f'day{DAY}.txt'

data = DATA_FILE.read_text()

## Solution

In [36]:
from typing import Tuple, Optional, NamedTuple, List
import re

test = '''
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
'''

class Dir(NamedTuple):
    name: str
    files: List[Tuple[str, int]]
    dirs: List['Dir']
    parent: Optional['Dir']

    def __str__(self) -> str:
        n = f' - {self.name}'
        f = '\n'.join([f'\t - {fn} ({s})' for fn, s in self.files])
        d = '\n'.join([str(dir) for dir in self.dirs])
        d = '\n'.join([f'\t{l}' for l in list(d.split('\n'))])
        return f'{n}\n{f}\n{d}'


root = Dir('/', [], [], None)


def add_dir(dir_name: str, curr_dir: Dir):
    try:
        return next(d for d in curr_dir.dirs if d.name == dir_name)
    except:
        new_dir = Dir(dir_name, [], [], curr_dir)
        curr_dir.dirs.append(new_dir)
        return new_dir

def cd(path: str, curr_dir: Dir) -> Dir:
    if path == '..':
        return curr_dir.parent
    elif path == '/':
        return root
    
    return add_dir(path, curr_dir)


def command(c: str, curr_dir: Dir):
    if c.startswith('cd '):
        return cd(c[3:], curr_dir)
    else:
        return curr_dir

file_matcher = re.compile('(\d+) (.+)')

top_curr_dir = root
for line in data.strip().split('\n'):
    if line.startswith('$ '):
        top_curr_dir = command(line[2:], top_curr_dir)
    elif line.startswith('dir '):
        add_dir(line[4:], top_curr_dir)
    else:
        size, name = file_matcher.match(line).groups()
        top_curr_dir.files.append((name, int(size)))

def part1(curr_dir: Dir):
    total = 0
    less_than_100k = []
    for dir in curr_dir.dirs:
        size, found = part1(dir)
        total += size
        less_than_100k.extend(found)
    
    for fname, fsize in curr_dir.files:
        total += fsize

    if total <= 100000:
        less_than_100k.append(total)
    
    return total, less_than_100k

root_size, all_less_than_100k = part1(root)
print('Part 1 Answer: ', sum(all_less_than_100k))

TOTAL_SPACE = 70000000
UNUSED_SPACE = 30000000

def part2(curr_dir: Dir, needed: int):
    total = 0
    smallest = root_size
    for dir in curr_dir.dirs:
        size, found = part2(dir, needed)
        total += size
        smallest = min(found, smallest)
    
    for fname, fsize in curr_dir.files:
        total += fsize

    if total >= needed:
        smallest = min(total, smallest)
    
    return total, smallest

root_size, smallest_needed = part2(root, UNUSED_SPACE - (TOTAL_SPACE - root_size))
print('Part 2 Answer: ', smallest_needed)

Part 1 Answer:  1583951
Part 2 Answer:  214171
